# Введение в обработку текста на естественном языке

Материалы:
* Макрушин С.В. Лекция 9: Введение в обработку текста на естественном языке\
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

## Задачи для совместного разбора

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
import pymorphy2


ModuleNotFoundError: No module named 'pymorphy2'

1. Считайте слова из файла `litw-win.txt` и запишите их в список `words`. В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка `words`. Считайте, что в слове есть опечатка, если данное слово не содержится в списке `words`. 

In [12]:
import Levenshtein


In [13]:
with open ("./data/litw-win.txt", "r", encoding='windows-1251') as file:
    words = [i.split()[-1] for i in file]
    
    
def closest_word_by_Levenshtein(word, words):
    closest = None
    closest_distance = float('inf')
    for w in words:
        distance = Levenshtein.distance(word, w)
        if distance < closest_distance:
            closest_distance = distance
            closest = w
    return closest


def correct_text(text, words):
    corrected = []
    for word in text.split():
        if word not in words:
            closest = closest_word_by_Levenshtein(word, words)
            corrected.append(closest)
        else:
            corrected.append(word)
    return ' '.join(corrected)


text = '''с велечайшим усилием выбравшись из потока убегающих людей Кутузов со свитой уменьшевшейся вдвое поехал на звуки выстрелов русских орудий'''
corrected = correct_text(text, words)
print(corrected)
print(text)

с величайшим усилием выбравшись из потока убегающих людей кутузов со свитой уменьшившейся вдвое поехал на звуки выстрелов русских орудий
с велечайшим усилием выбравшись из потока убегающих людей Кутузов со свитой уменьшевшейся вдвое поехал на звуки выстрелов русских орудий


2. Разбейте текст из формулировки задания 1 на слова; проведите стемминг и лемматизацию слов.

In [14]:
from pymystem3 import Mystem
from snowballstemmer import stemmer
m = Mystem()

In [15]:
words = [i for i in text.split()]

In [16]:
stemmer = stemmer('russian')
stemmed_words = [stemmer.stemWord(word) for word in words]
stemmed_words

['с',
 'велечайш',
 'усил',
 'выбра',
 'из',
 'поток',
 'убега',
 'люд',
 'Кутуз',
 'со',
 'свит',
 'уменьшевш',
 'вдво',
 'поеха',
 'на',
 'звук',
 'выстрел',
 'русск',
 'оруд']

In [17]:
lemmatized_words = [m.lemmatize(word)[0] for word in words]
lemmatized_words

['с',
 'велечайший',
 'усилие',
 'выбираться',
 'из',
 'поток',
 'убегать',
 'человек',
 'кутузов',
 'со',
 'свита',
 'уменьшевшийся',
 'вдвое',
 'поехать',
 'на',
 'звук',
 'выстрел',
 'русский',
 'орудие']

3. Преобразуйте предложения из формулировки задания 1 в векторы при помощи `CountVectorizer`.

In [18]:
from sklearn.feature_extraction.text import CountVectorizer

text = ["Считайте слова из файла litw-win.txt и запишите их в список words.",
        "В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна)к ним слова из списка words.",
        "Считайте, что в слове есть опечатка, если данное слово не содержится в списке words."
        ]

CV = CountVectorizer()

vectors = CV.fit_transform(text)

print(vectors.toarray())

print(CV.get_feature_names())

[[1 1 1 1 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 1 0]
 [0 0 0 1 1 1 0 0 0 1 1 0 1 1 0 1 1 0 1 0 1 1 1 1 2 0 0 1 0 1 0 0 0 0 0]
 [0 0 0 1 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 1 0 1 0 1 0 1 0 1]]
['litw', 'txt', 'win', 'words', 'ближайшие', 'все', 'данное', 'если', 'есть', 'заданном', 'заменив', 'запишите', 'из', 'исправьте', 'их', 'левенштейна', 'на', 'не', 'ним', 'опечатка', 'опечатками', 'опечатки', 'предложении', 'расстояния', 'слова', 'слове', 'слово', 'смысле', 'содержится', 'списка', 'списке', 'список', 'считайте', 'файла', 'что']


C:\Users\andre\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


## Лабораторная работа 9

### Расстояние редактирования

1.1 Загрузите предобработанные описания рецептов из файла `preprocessed_descriptions.csv`. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). 

In [19]:
import pandas as pd
from nltk.tokenize import word_tokenize

df = pd.read_csv('./data/preprocessed_descriptions.csv')

df['preprocessed_descriptions']=df['preprocessed_descriptions'].apply(str)
texts = list(df['preprocessed_descriptions'])

words = set()
for text in texts:
    words.update(word_tokenize(text))

words_all = list()
for text in texts:
    words_all.append(word_tokenize(text))
print(len(words),len(words_all))

32868 30000


In [ ]:
import pandas as pd
import nltk
# Загрузка данных
df = pd.read_csv('./data/preprocessed_descriptions.csv')
df['preprocessed_descriptions']=df['preprocessed_descriptions'].apply(str)
# Список всех слов
all_words = []
for i in df['preprocessed_descriptions']:
    words = nltk.word_tokenize(i)
    all_words += words

# Набор уникальных слов
words = set(all_words)

print(f'Количество уникальных слов: {len(words)}')
print(f'Общее количество слов: {len(all_words)}')

1.2 Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние редактирования.

In [21]:
import random
import editdistance

pairs = random.sample(list(words), 10)

for i in range(0,len(pairs),2):
    pair1,pair2 = pairs[i],pairs[i+1]
    distance = editdistance.eval(pair1, pair2)
    print("Расстояние между {} и {}: {}".format(pair1, pair2, distance))

Расстояние между connoisseurs и selfraising: 11
Расстояние между frittatasi и havnt: 9
Расстояние между nava и budshese: 8
Расстояние между associate и stacks: 7
Расстояние между saucemakes и alfassia: 9


1.3 Напишите функцию, которая для заданного слова `word` возвращает `k` ближайших к нему слов из списка `words` (близость слов измеряется с помощью расстояния Левенштейна)

In [22]:
def get_k_nearest_words(word, words, k):
    distances = [(w, editdistance.eval(word, w)) for w in words]
    distances.sort(key=lambda x: x[1])
    return [w[0] for w in distances[:k]]


get_k_nearest_words("cool", words, 5)

['cool', 'coop', 'cook', 'col', 'chol']

### Стемминг, лемматизация

2.1 На основе результатов 1.1 создайте `pd.DataFrame` со столбцами: 
    * word
    * stemmed_word 
    * normalized_word 

Столбец `word` укажите в качестве индекса. 

Для стемминга воспользуйтесь `SnowballStemmer`, для нормализации слов - `WordNetLemmatizer`. Сравните результаты стемминга и лемматизации.

In [23]:
import pandas as pd
import nltk
from nltk.stem import SnowballStemmer, WordNetLemmatizer

stemmer = SnowballStemmer("english")
lemmatizer = WordNetLemmatizer()


def stem_word(word):
    return stemmer.stem(word)

def lem_word(word):
    return lemmatizer.lemmatize(word,"v")


df = pd.DataFrame(words, columns=['word'])
df['stemmed_word'] = df['word'].apply(stem_word)
df['normalized_word'] = df['word'].apply(lem_word)

df.set_index('word', inplace=True)

print(df)


             stemmed_word normalized_word
word                                     
johnny             johnni          johnny
falls                fall            fall
fear                 fear            fear
ilanna             ilanna          ilanna
heartwarming    heartwarm    heartwarming
...                   ...             ...
thailand         thailand        thailand
sopressata     sopressata      sopressata
batters            batter          batter
crannies           cranni        crannies
deteriorated     deterior     deteriorate

[32868 rows x 2 columns]


2.2. Удалите стоп-слова из описаний рецептов. Какую долю об общего количества слов составляли стоп-слова? Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

In [24]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from collections import Counter


stop_words = set(stopwords.words('english'))

# вычисление доли стоп-слов
all_words_filtered = [word for word in all_words if word not in stop_words]
print("До {}. После {}. Доля {}.".format(len(all_words), len(all_words_filtered),round(len(all_words_filtered)/len(all_words)*100,2)))



print("\nтоп-10 слов до удаления стоп-слов\n")
counter_all = Counter(all_words)
top_10 = counter_all.most_common(10)
k = 1
for word, count in top_10:
    print("{}. {}".format(k,word))
    k+=1
print("\nтоп-10 слов после удаления стоп-слов\n")
counter_all_fil = Counter(all_words_filtered)
top_10_fil = counter_all_fil.most_common(10)
k = 1
for word, count in top_10_fil:
    print("{}. {}".format(k,word))
    k+=1

До 1069885. После 581520. Доля 54.35.

топ-10 слов до удаления стоп-слов

1. the
2. a
3. and
4. this
5. i
6. to
7. is
8. it
9. of
10. for

топ-10 слов после удаления стоп-слов

1. recipe
2. make
3. time
4. use
5. great
6. like
7. easy
8. one
9. made
10. good


### Векторное представление текста

3.1 Выберите случайным образом 5 рецептов из набора данных. Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

# загрузка данных из файла
data = pd.read_csv('./data/preprocessed_descriptions.csv')
data['preprocessed_descriptions']=data['preprocessed_descriptions'].apply(str)
# выбор 5 случайных рецептов
random_recipes = data.sample(5)

# создание объекта TfidfVectorizer
vectorizer = TfidfVectorizer()

# преобразование описания каждого рецепта в числовой вектор
for i, row in random_recipes.iterrows():
    description = row['preprocessed_descriptions']
    vector = vectorizer.fit_transform([description])
    print(f"Рецепт {i+1}:\n{description}\nВектор:\n{vector.toarray()}\n")

Рецепт 1517:
given by a friend and i find the difference in this dip enjoyable
Вектор:
[[0.30151134 0.30151134 0.30151134 0.30151134 0.30151134 0.30151134
  0.30151134 0.30151134 0.30151134 0.30151134 0.30151134]]

Рецепт 1075:
light summer dessert
Вектор:
[[0.57735027 0.57735027 0.57735027]]

Рецепт 16243:
i have been making this for years  it is very tasty chicken recipe and one that isnt to be missed try it and see for yourself all the great mix of flavors in this tender juicy chicken
Вектор:
[[0.14744196 0.29488391 0.14744196 0.14744196 0.29488391 0.14744196
  0.29488391 0.14744196 0.14744196 0.14744196 0.14744196 0.14744196
  0.29488391 0.14744196 0.14744196 0.14744196 0.14744196 0.14744196
  0.14744196 0.14744196 0.14744196 0.14744196 0.14744196 0.14744196
  0.14744196 0.29488391 0.14744196 0.14744196 0.14744196 0.14744196
  0.14744196]]

Рецепт 4323:
this is a tasty filling and inexpensive dishfrom the lone star cookbook 199798 aggie moms club texarkana tx
Вектор:
[[0.24253563 0

3.2 Вычислите близость между каждой парой рецептов, выбранных в задании 3.1, используя косинусное расстояние (`scipy.spatial.distance.cosine`) Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

In [26]:
import scipy
from sklearn.metrics.pairwise import cosine_distances

In [33]:
X = vectorizer.fit_transform(data['preprocessed_descriptions'])

# получаем матрицу векторов
matrix = X.toarray()

# вычисляем близость между каждой парой рецептов
distances = cosine_distances(matrix[random_recipes.index])

# формируем DataFrame с результатами
result_df = pd.DataFrame(distances, index=random_recipes.index, columns=random_recipes.index)
result_df.columns.name = 'Recipes'
result_df.index.name = 'Recipes'
result_df

Recipes,1516,1074,16242,4322,10426
Recipes,,,,,
1516,0.000000,1.0,0.956913,0.984280,0.988199
1074,1.000000,0.0,1.000000,1.000000,1.000000
16242,0.956913,1.0,0.000000,0.954687,0.953010
4322,0.984280,1.0,0.954687,0.000000,0.989145
10426,0.988199,1.0,0.953010,0.989145,0.000000


3.3 Какие рецепты являются наиболее похожими? Прокомментируйте результат (словами).

In [29]:
Рецепт 1074 и 16242

,Unnamed: 0,name,preprocessed_descriptions
0,0,george s at the cove black bean soup,an original recipe created by chef scott meska...
1,1,healthy for them yogurt popsicles,my children and their friends ask for my homem...
2,2,i can t believe it s spinach,these were so go it surprised even me
3,3,italian gut busters,my sisterinlaw made these for us at a family g...
4,4,love is in the air beef fondue sauces,i think a fondue is a very romantic casual din...
...,...,...,...
29995,29995,zurie s holey rustic olive and cheddar bread,this is based on a french recipe but i changed...
29996,29996,zwetschgenkuchen bavarian plum cake,this is a traditional fresh plum cake thought ...
29997,29997,zwiebelkuchen southwest german onion cake,this is a traditional late summer early fall s...
29998,29998,zydeco soup,this is a delicious soup that i originally fou...
